# Исследование популярности компьютерных игр

Вы работаете в интернет-магазине «Стримчик», который продаёт по всему миру компьютерные игры. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы (например, Xbox или PlayStation). Вам нужно выявить определяющие успешность игры закономерности. Это позволит сделать ставку на потенциально популярный продукт и спланировать рекламные кампании.


Перед вами данные до 2016 года. Представим, что сейчас декабрь 2016 г., и вы планируете кампанию на 2017-й. Нужно отработать принцип работы с данными. Неважно, прогнозируете ли вы продажи на 2017 год по данным 2016-го или же 2027-й — по данным 2026 года.


В наборе данных попадается аббревиатура ESRB (Entertainment Software Rating Board) — это ассоциация, определяющая возрастной рейтинг компьютерных игр. ESRB оценивает игровой контент и присваивает ему подходящую возрастную категорию, например, «Для взрослых», «Для детей младшего возраста» или «Для подростков».

## Инструкция по выполнению проекта
Шаг 1. Откройте файл с данными и изучите общую информацию
Путь к файлу: /datasets/games.csv. Скачать датасет  

Шаг 2. Подготовьте данные  
- [x] Замените названия столбцов (приведите к нижнему регистру);  
- [x] Преобразуйте данные в нужные типы. Опишите, в каких столбцах заменили тип данных и почему;
- [x] Обработайте пропуски при необходимости:
- [x] Объясните, почему заполнили пропуски определённым образом или почему не стали это делать;
- [x] Опишите причины, которые могли привести к пропускам;
- [x] Обратите внимание на аббревиатуру 'tbd' в столбцах с рейтингом. Отдельно разберите это значение и опишите, как его обработать;
- [x] Посчитайте суммарные продажи во всех регионах и запишите их в отдельный столбец.  

Шаг 3. Проведите исследовательский анализ данных  
- Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?
- Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
- Возьмите данные за соответствующий актуальный период. Актуальный период определите самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год. Не учитывайте в работе данные за предыдущие годы.
- Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.
- Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.
- Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы.
- Соотнесите выводы с продажами игр на других платформах.
- Посмотрите на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?  

Шаг 4. Составьте портрет пользователя каждого региона  
- Определите для пользователя каждого региона (NA, EU, JP):
- Самые популярные платформы (топ-5). Опишите различия в долях продаж.
- Самые популярные жанры (топ-5). Поясните разницу.
- Влияет ли рейтинг ESRB на продажи в отдельном регионе?  

Шаг 5. Проверьте гипотезы  
- Средние пользовательские рейтинги платформ Xbox One и PC одинаковые;
- Средние пользовательские рейтинги жанров Action (англ. «действие», экшен-игры) и Sports (англ. «спортивные соревнования») разные.
- Задайте самостоятельно пороговое значение alpha.
- Поясните: 
   - Как вы сформулировали нулевую и альтернативную гипотезы; 
   - Какой критерий применили для проверки гипотез и почему.  
   
Шаг 6. Напишите общий вывод  

### Описание данных
`Name` — название игры  
`Platform` — платформa  
`Year_of_Release` — год выпуска  
`Genre` — жанр игры  
`NA_sales` — продажи в Северной Америке (миллионы проданных копий)  
`EU_sales` — продажи в Европе (миллионы проданных копий)  
`JP_sales`— продажи в Японии (миллионы проданных копий)  
`Other_sales` — продажи в других странах (миллионы проданных копий)  
`Critic_Score` — оценка критиков (максимум 100)  
`User_Score` — оценка пользователей (максимум 10)  
`Rating` — рейтинг от организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.  

---

Данные за 2016 год могут быть неполными.  

Как будут проверять мой проект?
Мы подготовили критерии оценки проекта. Прежде чем решать кейс, внимательно изучите их.
На что обращают внимание при проверке проекта:
Как вы описываете выявленные в данных проблемы?
Как готовите датасет к анализу?
Какие строите графики для распределений и как их объясняете?
Как рассчитываете стандартное отклонение и дисперсию?
Формулируете ли альтернативную и нулевую гипотезы?
Какие методы применяете, чтобы их проверить?
Объясняете результат проверки гипотезы или нет?
Соблюдаете ли структуру проекта и поддерживаете аккуратность кода?
Какие выводы делаете?
Оставляете ли комментарии к шагам?




### 1. Подготовка данных. 

In [76]:
import pandas as pd
import math
import matplotlib.pyplot as plt 
import plotly.express as px
import numpy as np
from scipy import stats as st

In [77]:
games = pd.read_csv('datasets/games.csv') # добавить '/'' перед отправкой!!!

In [78]:
games.head(10)

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


Приведем названия столбцов к нижнему регистру для удобства:

In [79]:
games.columns = games.columns.str.lower()
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

In [80]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
name               16713 non-null object
platform           16715 non-null object
year_of_release    16446 non-null float64
genre              16713 non-null object
na_sales           16715 non-null float64
eu_sales           16715 non-null float64
jp_sales           16715 non-null float64
other_sales        16715 non-null float64
critic_score       8137 non-null float64
user_score         10014 non-null object
rating             9949 non-null object
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [81]:
games.critic_score.describe()

count    8137.000000
mean       68.967679
std        13.938165
min        13.000000
25%        60.000000
50%        71.000000
75%        79.000000
max        98.000000
Name: critic_score, dtype: float64

In [82]:
#количество пропусков
total = games.isnull().sum().sort_values(ascending=False)
percent = ((games.isnull().sum()/games.isnull().count())*100).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.style.format('{:.1f}', subset='Percent')

,Total,Percent
critic_score,8578,51.3
rating,6766,40.5
user_score,6701,40.1
year_of_release,269,1.6
genre,2,0.0
name,2,0.0
other_sales,0,0.0
jp_sales,0,0.0
eu_sales,0,0.0
na_sales,0,0.0


In [83]:
display(len(games['name'].unique()))
games[games['name'].isnull()]

11560

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


В столбце 'name' всего 2 пропущенных значения и они совпадают с пропусками в столбце 'genre'. Можем безболезненно от них избавиться, так как в этих строках отсутствуют и все рейтинги:

In [84]:
games.drop([659, 14244], inplace=True)

Проверим нет ли ошибок или повторов в названиях жанров:

In [85]:
games['genre'] = games['genre'].str.lower()
games['genre'].unique()

array(['sports', 'platform', 'racing', 'role-playing', 'puzzle', 'misc',
       'shooter', 'simulation', 'action', 'fighting', 'adventure',
       'strategy'], dtype=object)

Проверим, есть ли дубликаты:

In [86]:
print('Полных дубликатов: ',games.duplicated().sum())

# в графе 'name' могут быть ошибки, одни и те же буквы в разном регистре и т.п., 
# поэтому проверим на дубликаты еще и не учитывая этот столбец:
print('Дубликатов по всем столбцам, кроме "name": ', games.duplicated(subset=games.columns[games.columns != 'name'],keep=False).sum())

print('Critic score ',games[games.duplicated(subset=games.columns[games.columns != 'name'],keep=False)]['critic_score'].unique())
print('User score ',games[games.duplicated(subset=games.columns[games.columns != 'name'],keep=False)]['user_score'].unique())



Полных дубликатов:  0
Дубликатов по всем столбцам, кроме "name":  1568
Critic score  [nan]
User score  [nan 'tbd']


Выявленные дубликаты имеют пропуски в `critic_score` и `user_score`, если бы в этих столбцах были бы идентичные значения, мы бы признали их дубликатами. 

In [87]:
games['year_of_release'].describe()

count    16444.000000
mean      2006.486256
std          5.875525
min       1980.000000
25%       2003.000000
50%       2007.000000
75%       2010.000000
max       2016.000000
Name: year_of_release, dtype: float64

Год выпуска `year_of_release` из формата с плавающей точкой приведем к формату *int64* так как это всегда целое число; `user_score` из строки - к *float64* с помощью *to_numeric*, т.к. это числовое значение. В столбце `user_score` есть значения 'tbd' ('to be determined'), что означает, что у игры еще недостаточно оценок для определения средней, заменим эти значения на NaN с помощью параметра errors='coerce'. `critic_score` из *float64* приведем к *int64*, так как это целые числа:

In [88]:
display(len(games.query('user_score=="tbd"')))
games['year_of_release'] = games['year_of_release'].astype('Int64')
games['critic_score'] = games['critic_score'].astype('Int64')
games['user_score'] = pd.to_numeric(games['user_score'], errors='coerce')

2424

In [89]:
games.info()
games

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
name               16713 non-null object
platform           16713 non-null object
year_of_release    16444 non-null Int64
genre              16713 non-null object
na_sales           16713 non-null float64
eu_sales           16713 non-null float64
jp_sales           16713 non-null float64
other_sales        16713 non-null float64
critic_score       8137 non-null Int64
user_score         7590 non-null float64
rating             9949 non-null object
dtypes: Int64(2), float64(5), object(4)
memory usage: 1.6+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E
1,Super Mario Bros.,NES,1985,platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,role-playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016,action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16711,LMA Manager 2007,X360,2006,sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16712,Haitaka no Psychedelica,PSV,2016,adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16713,Spirits & Spells,GBA,2003,platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


In [90]:
games[games['critic_score'].isna()]['year_of_release'].describe()

count    8461.000000
mean     2005.819643
std         7.044294
min      1980.000000
25%      2001.000000
50%      2008.000000
75%      2011.000000
max      2016.000000
Name: year_of_release, dtype: float64

In [91]:
games[games['rating'].isna()]['year_of_release'].describe()

count    6676.000000
mean     2004.883463
std         7.491332
min      1980.000000
25%      1999.000000
50%      2007.000000
75%      2011.000000
max      2016.000000
Name: year_of_release, dtype: float64

In [92]:
def total_sales(row):
    return row['na_sales'] +row['eu_sales'] + row['jp_sales'] + row['other_sales']

games['total_sales'] = games.apply(total_sales, axis=1)
games.head(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E,82.54
1,Super Mario Bros.,NES,1985,platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996,role-playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38
5,Tetris,GB,1989,puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN,30.26
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.5,E,29.80
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58,6.6,E,28.91
8,New Super Mario Bros. Wii,Wii,2009,platform,14.44,6.94,4.70,2.24,87,8.4,E,28.32
9,Duck Hunt,NES,1984,shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN,28.31


In [94]:
games.query('year_of_release < 1994')['rating'].value_counts()

T       2
M       1
E10+    1
Name: rating, dtype: int64

### Вывод
Пропуски в оценках пользователя, критиков и в рейтинге не были заменены, так как это важные показатели для нашего анализа. Данные могли быть пропущены из-за небольшого количества оценок. Пропуски в рейтинге ESRB в старых играх скорее всего из-за того, что система ESRB появилась только в 1994. Пропуски в годах будут проигнорированы, скорей всего это была недостающая информация при парсинге данных или откуда брали данные о играх. Из названий некоторых игр можно "достать" год выпуска, но таких игр немного. В столбцах жанра и имени были 2 пропуска в одних и тех же записях - они были удалены из таблицы. 

Год и оценка критиков были приведены к целочисленному значению, оценка пользователей была приведена к формату с плавающей точкой, *'tbd'* были заменены на NaN.

Был добавлен столбец `total_sales` с суммой продаж по всему миру.

### 2. Исследовательский анализ данных.

- [x] Посмотрите, сколько игр выпускалось в разные годы. Важны ли данные за все периоды?
- [x] Посмотрите, как менялись продажи по платформам. Выберите платформы с наибольшими суммарными продажами и постройте распределение по годам. За какой характерный срок появляются новые и исчезают старые платформы?
- [x] Возьмите данные за соответствующий актуальный период. Актуальный период определите самостоятельно в результате исследования предыдущих вопросов. Основной фактор — эти данные помогут построить прогноз на 2017 год. Не учитывайте в работе данные за предыдущие годы.
- Какие платформы лидируют по продажам, растут или падают? Выберите несколько потенциально прибыльных платформ.
- Постройте график «ящик с усами» по глобальным продажам игр в разбивке по платформам. Опишите результат.
- Посмотрите, как влияют на продажи внутри одной популярной платформы отзывы пользователей и критиков. Постройте диаграмму рассеяния и посчитайте корреляцию между отзывами и продажами. Сформулируйте выводы.
- Соотнесите выводы с продажами игр на других платформах.
- Посмотрите на общее распределение игр по жанрам. Что можно сказать о самых прибыльных жанрах? Выделяются ли жанры с высокими и низкими продажами?  



In [159]:
numb_games_yearly = games.pivot_table(index='year_of_release', values='name', aggfunc='count').reset_index()
fig = px.line(numb_games_yearly, x='year_of_release', y='name', title='Количество выпущенных игр в год')
fig.show()



Начиная с 1994 года наблюдается рост количества выпускаемых игр, в нашем анализе не будем использовать информацию раньше этого года:

In [160]:
games_new = games.query('year_of_release >= 1994')

In [161]:
games_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16079 entries, 0 to 16714
Data columns (total 12 columns):
name               16079 non-null object
platform           16079 non-null object
year_of_release    16079 non-null Int64
genre              16079 non-null object
na_sales           16079 non-null float64
eu_sales           16079 non-null float64
jp_sales           16079 non-null float64
other_sales        16079 non-null float64
critic_score       7980 non-null Int64
user_score         7460 non-null float64
rating             9764 non-null object
total_sales        16079 non-null float64
dtypes: Int64(2), float64(6), object(4)
memory usage: 1.6+ MB


In [162]:
top10_platforms = games_new.groupby('platform')['total_sales'].sum().sort_values(ascending=False)[:10]

In [163]:
display(top10_platforms.index) # список названий 10 топовых платформ
top10_df = games_new.query('platform in @top10_platforms.index')
top10_df

Index(['PS2', 'X360', 'PS3', 'Wii', 'DS', 'PS', 'PS4', 'GBA', 'PSP', '3DS'], dtype='object', name='platform')

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76,8.0,E,82.54
2,Mario Kart Wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80,8.0,E,32.77
6,New Super Mario Bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89,8.5,E,29.80
7,Wii Play,Wii,2006,misc,13.96,9.18,2.93,2.84,58,6.6,E,28.91
...,...,...,...,...,...,...,...,...,...,...,...,...
16708,Woody Woodpecker in Crazy Castle 5,GBA,2002,platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN,0.01
16709,SCORE International Baja 1000: The Official Game,PS2,2008,racing,0.00,0.00,0.00,0.00,NaN,NaN,NaN,0.00
16710,Samurai Warriors: Sanada Maru,PS3,2016,action,0.00,0.00,0.01,0.00,NaN,NaN,NaN,0.01
16711,LMA Manager 2007,X360,2006,sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN,0.01


In [167]:
fig_data = top10_df.pivot_table(index=['platform', 'year_of_release'], values='total_sales', aggfunc='sum').reset_index()
fig = px.line(fig_data, x='year_of_release', y='total_sales', color='platform', title='Суммарные продажи топ-10 платформ')
fig.show()
box_fig = px.histogram(top10_df, x='year_of_release', y='total_sales', color='platform', barmode='overlay', marginal='box', title='Суммарные продажи топ-10 платформ')
box_fig.show()

In [166]:
data = top10_df.pivot_table(index=['platform'], values='year_of_release', aggfunc=['min', 'max']).reset_index()
data.columns=data.columns.droplevel(1)
data['lifetime'] = data['max'] - data['min']
last_year = max(data['max'])
# при подсчете средней продолжительности жизни платформы не будем учитывать те платформы, которые на момент 2016 года еще существуют
print(f'Средняя продолжительность жизни платформы {data.query("max<@last_year")["lifetime"].mean()}')


Средняя продолжительность жизни платформы 9.4
4.581363212940979 5.478239782184704
